# Generate webpage content for calculation_system_relax records

This Notebook is designed for reading finished calculation_system_relax records and generating the associated webpage content.

#### Library imports

In [1]:
# Standard Python libraries
from __future__ import print_function
import glob
import os
from collections import OrderedDict
from copy import deepcopy

from IPython.core.display import display, HTML

# pandas.pydata.org
import pandas as pd

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

#### Plotting library imports

In [2]:
# https://bokeh.pydata.org/
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.resources import Resources, CDN
from bokeh.io import output_notebook
from bokeh.models import Range1d
print('bokeh version =', bokeh.__version__)
output_notebook()

bokeh version = 0.12.4


Loading BokehJS ...

## 1. Read Calculation Data

This section reads in raw data from a database. 

## 1. Raw Data

This section reads in or generates the raw_data associated with the calculation. 

### 1.1 Initialize database

- __dbasename__ is used here to predefine different dbase settings
- __dbase__ is the iprPy.Database object to use for accessing a database

In [3]:
dbasename = 'test'

# 'local' is a local directory
if   dbasename == 'local':
    dbase = iprPy.Database('local',   host='C:\Users\lmh1\Documents\calculations\ipr\library')

# 'test' is a local directory for testing 
if   dbasename == 'test':
    dbase = iprPy.Database('local',   host='C:\Users\lmh1\Documents\calculations\ipr\library_test')
    
# 'curator' is a local MDCS curator
elif dbasename == 'curator':
    dbase = iprPy.Database('curator', host='http://127.0.0.1:8000/', 
                                      user='admin', 
                                      pswd='admin')

# 'iprhub' is the remote MDCS curator at iprhub
elif dbasename == 'iprhub':
    dbase = iprPy.Database('curator', host='https://iprhub.nist.gov/', 
                                      user='lmh1',
                                      pswd='C:/users/lmh1/documents/iprhub/iprhub_password.txt',
                                      cert='C:/users/lmh1/documents/iprhub/iprhub-ca.pem')
else:
    raise ValueError('unknown dbasename ' + dbasename)

### 1.2 Access records

In [4]:
proto_df = dbase.get_records_df(style='crystal_prototype')
print(str(len(proto_df)) + ' prototype records loaded')

19 prototype records loaded


In [5]:
pot_df = dbase.get_records_df(style='potential_LAMMPS')
print(str(len(pot_df)) + ' potential records loaded')

149 potential records loaded


In [6]:
raw_df = dbase.get_records_df(style='calculation_system_relax')
print(str(len(raw_df)) + ' calculation records loaded')

452 calculation records loaded


### 1.3 Check errors

In [7]:
if 'error' in raw_df:
    for error in np.unique(raw_df[pd.notnull(raw_df.error)].error):
        print(error)
        print()

Traceback (most recent call last):
  File "calc_refine_structure.py", line 444, in <module>
    main(*sys.argv[1:])
  File "calc_refine_structure.py", line 52, in main
    strainrange = input_dict['strainrange'])
  File "calc_refine_structure.py", line 158, in quick_a_Cij
    strainrange=strainrange, cycle=cycle)
  File "calc_refine_structure.py", line 365, in calc_cij
    raise RuntimeError('Divergence of box dimensions to <= 0')
RuntimeError: Divergence of box dimensions to <= 0

Traceback (most recent call last):
  File "calc_LAMMPS_ELASTIC.py", line 450, in <module>
    main(*sys.argv[1:])
  File "calc_LAMMPS_ELASTIC.py", line 58, in main
    pressure_unit = input_dict['pressure_unit'])
  File "calc_LAMMPS_ELASTIC.py", line 170, in lammps_ELASTIC_refine
    dmax=dmax, pressure_unit=pressure_unit)
  File "calc_LAMMPS_ELASTIC.py", line 333, in lammps_ELASTIC
    output = lmp.run(lammps_command, 'in.elastic', mpi_command)
un.py", line 98, in run
    raise ValueError('Invalid LAMMPS in

## 2. Process Data

This section processes and refines the data.

### 2.1 Identify composition

We need to identify the composition of each calculation so that we can collect duplicates and filter out artificial compounds.

- __counts__ is a dictionary counting the number of times each atype appears in a crystal prototype's unit cell (i.e. the number of symmetry equivalent sites)

In [8]:
counts = {}
for i, prototype in proto_df.iterrows():
    model = DM(dbase.get_record(name=prototype.id).content)
    counts[prototype.id] = np.unique(model.finds('component'), return_counts=True)[1]

- __comp_refine()__ takes a list of symbols and count of how many times each symbol appears in a structure and generates a composition string.__comp_refine__ takes a list of symbols and count of how many times each symbol appears in a structure and generates a composition string.

In [9]:
def comp_refine(symbols, counts):
    """Takes a list of symbols and count of how many times each symbol appears and generates a composition string."""
    primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47]
    
    sym_dict = {}
    for i in xrange(len(symbols)):
        sym_dict[symbols[i]] = counts[i]
    
    for prime in primes:
        if max(sym_dict.values()) < prime:
            break
        
        while True:
            breaktime = False
            for value in sym_dict.values():
                if value % prime != 0:
                    breaktime = True
                    break
            if breaktime:
                break
            for key in sym_dict:
                sym_dict[key] /= prime
    
    composition=''
    for key in sorted(sym_dict):
        if sym_dict[key] > 0:
            composition += key
            if sym_dict[key] != 1:
                composition += str(sym_dict[key])
            
    return composition       

In [10]:
compositions = []
for i, calc in raw_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.family]))
raw_df = raw_df.assign(composition=compositions)

### 2.2 Identify current ipr potentials 

In [11]:
# Extract versionstyle and versionnumber from potential implementation ids
versionstyle = []
versionnumber = []
for name in pot_df['id'].values:
    version = name.split('--')[-1]
    try:
        versionnumber.append(int(version[-1]))
    except:
        versionnumber.append(np.nan)
        versionstyle.append(version)
    else:
        versionstyle.append(version[:-1])

pot_df['versionstyle'] = versionstyle
pot_df['versionnumber'] = versionnumber

# Loop through unique potential id's
includeid = []
for pot_id in np.unique(pot_df.pot_id.values):
    check_df = pot_df[pot_df.pot_id == pot_id]
    check_df = check_df[check_df.versionstyle == 'ipr']
    check_df = check_df[check_df.versionnumber == check_df.versionnumber.max()]
    if len(check_df) == 1:
        includeid.append(check_df['id'].values[0])
    elif len(check_df) > 1:
        raise ValueError('Bad currentIPR check for '+pot_id)

# Identify current IPR potentials
raw_df['currentIPR'] = raw_df.potential_LAMMPS_id.isin(includeid)

### 2.3 Identify crystals that have relaxed to a different crystal family

In [12]:
tol = 1e-5

In [13]:
crystal_families = {}
for proto_id, crystal_family in zip(proto_df['id'], proto_df.crystal_family):
    crystal_families[proto_id] = crystal_family
    
crystal_families = {
    #elemental
    'A1--Cu--fcc':                'cubic',
    'A2--W--bcc':                 'cubic',
    'A3--Mg--hcp':                'hexagonal',
    'A3\'--alpha-La--double-hcp': 'hexagonal',
    'A4--C--dc':                  'cubic',
    'A5--beta-Sn':                'tetragonal',
    'A6--In--bct':                'tetragonal',
    'A7--alpha-As':               'hexagonal',
    'A15--beta-W':                'cubic',
    'Ah--alpha-Po--sc':           'cubic',
   #1:1
    'B1--NaCl--rock-salt':        'cubic',
    'B2--CsCl':                   'cubic',
    'B3--ZnS--cubic-zinc-blende': 'cubic',
    'L1_0--AuCu':                 'tetragonal',
   #1:2
    'C1--CaF2--fluorite':         'cubic',
   #1:3
    'A15--Cr3Si':                 'cubic',
    'D0_3--BiF3':                 'cubic',
    'L1_2--AuCu3':                'cubic',
   #1:1:2
    'L2_1--AlCu2Mn--heusler':     'cubic'
}
np.unique(crystal_families.values())

array(['cubic', 'hexagonal', 'tetragonal'], 
      dtype='|S10')

In [14]:
hasrelaxed = np.empty(len(raw_df), dtype=bool)
for i, row in enumerate(raw_df.itertuples()):
    
    # Extract data for comparison
    a = row.final_a
    b = row.final_b
    c = row.final_c
    family = row.family
    crystal_family = crystal_families[family]
    
    # Evaluate based on crystal_family
    if crystal_family == 'cubic':
        if np.isclose(b/a, 1.0, atol=tol, rtol=0.0) and np.isclose(c/a, 1.0, atol=tol, rtol=0.0):
            hasrelaxed[i] = False
        else:
            hasrelaxed[i] = True
 
    elif crystal_family == 'hexagonal':
        if np.isclose(b/a, 3.**0.5, atol=tol, rtol=0.0):
            hasrelaxed[i] = False
        else:
            hasrelaxed[i] = True
            
    elif crystal_family == 'tetragonal':
        if np.isclose(b/a, 1.0, atol=tol, rtol=0.0) and not np.isclose(c/a, 1.0, atol=tol, rtol=0.0):
            hasrelaxed[i] = False
        else:
            hasrelaxed[i] = True

    # Evaluate based on family
    if family == 'A6--In--bct':
        if np.isclose(c/a, 2**(0.5), atol=tol, rtol=0.0):
            hasrelaxed[i] = True
    elif family == 'L1_0--AuCu':
        if np.isclose(c/a, 2**(0.5)/2, atol=tol, rtol=0.0):
            hasrelaxed[i] = True
            
raw_df['hasrelaxed'] = hasrelaxed

### 2.4 Remove unwanted calculations

Here is where we filter out unwanted entries (i.e. rows).

- __df__ is the dataframe during/after processing and refining

In [15]:
raw_df.keys()

Index([u'C', u'E_cohesive', u'LAMMPS_version', u'calc_key', u'calc_script',
       u'error', u'family', u'final_a', u'final_b', u'final_c', u'initial_a',
       u'initial_b', u'initial_c', u'iprPy_version', u'load_file',
       u'load_options', u'load_style', u'potential_LAMMPS_id',
       u'potential_LAMMPS_key', u'potential_id', u'potential_key',
       u'pressure_xx', u'pressure_yy', u'pressure_zz', u'sizemults', u'status',
       u'strainrange', u'symbols', u'temperature', u'composition',
       u'currentIPR', u'hasrelaxed'],
      dtype='object')

In [16]:
df = deepcopy(raw_df)

# Ignore unfinished or error calculations
df = df[df.status == 'finished']

# Ignore any implementations that are not current IPR implementations
df = df[df.currentIPR == True]

# Ignore any calculations that relaxed to a different crystal family
df = df[df.hasrelaxed == False]

# Ignore any that don't use the standard run parameters
df = df[np.isclose(df.strainrange, 1e-8, atol=1e-10, rtol=0.0) 
      & np.isclose(df.pressure_xx, 0.0, atol=1e-10, rtol=0.0)
      & np.isclose(df.pressure_yy, 0.0, atol=1e-10, rtol=0.0)
      & np.isclose(df.pressure_zz, 0.0, atol=1e-10, rtol=0.0)]

# Ignore false compounds (where # of unique symbols != # of symbols)
df = df[df.symbols.apply(lambda x: len(np.unique(x))) == df.symbols.apply(lambda x: len(x))] 

# Ignore duplicate compounds
ignore = set()
for i in xrange(len(df)):
    trunc = df.iloc[i+1:]
    matches = trunc.calc_key[  (trunc.potential_id == df.iloc[i].potential_id) 
                             & (trunc.family == df.iloc[i].family) 
                             & (trunc.composition == df.iloc[i].composition)
                             & (trunc.calc_script == df.iloc[i].calc_script)
                             & np.isclose(trunc.E_cohesive, df.iloc[i].E_cohesive, atol=1e-6, rtol=0.0)
                             & np.isclose(trunc.final_a, df.iloc[i].final_a, atol=1e-6, rtol=0.0)
                             & np.isclose(trunc.final_b, df.iloc[i].final_b, atol=1e-6, rtol=0.0)
                             & np.isclose(trunc.final_c, df.iloc[i].final_c, atol=1e-6, rtol=0.0)
                            ].tolist()
    ignore = ignore.union(matches)
df = df[~df.calc_key.isin(ignore)]

df.reset_index(drop=True, inplace=True)
print(str(len(df)) + ' records after filtering')

129 records after filtering


In [17]:
np.isclose([1,2,3,4,5], 3)

array([False, False,  True, False, False], dtype=bool)

### 2.5 Filter out extra data

Here, we limit the DataFrame to only the data that we care about (i.e. columns).

- __headers__ gives the list of data columns from raw_data to include in and how they should be renamed in data.

In [18]:
#                        raw names       new names
headers = OrderedDict([ ('calc_script',  'calculation'),
                        ('potential_id', 'potential'  ),
                        ('family',       'family'     ),
                        ('composition',  'composition'),
                        ('E_cohesive',   'Ecoh'),
                        ('final_a',      'alat'),
                        ('final_b',      'blat'),
                        ('final_c',      'clat'),
                        ('C',            'C'),
                      ])

df = pd.DataFrame(df, columns=headers.keys())
df.rename(columns=headers, inplace=True)
df

,calculation,potential,family,composition,Ecoh,alat,blat,clat,C
0,calc_refine_structure,2009--Purja-Pun-G-P--Ni-Al,B3--ZnS--cubic-zinc-blende,AlNi,-3.248053,5.305210,5.305210,5.305210,[[ 0.15202459 0.33277116 0.33277116 0. ...
1,calc_LAMMPS_ELASTIC,2012--Park-H--Mo,A7--alpha-As,Mo,-6.817378,4.479399,7.758546,5.486120,[[ 2.36345341 0.98600666 1.07883062 0. ...
2,calc_LAMMPS_ELASTIC,2009--Purja-Pun-G-P--Ni-Al,A3'--alpha-La--double-hcp,Al,-3.343863,2.845808,4.929084,9.534411,[[ 0.73365243 0.36331407 0.32538723 0. ...
3,calc_refine_structure,2012--Park-H--Mo,A5--beta-Sn,Mo,-6.121625,4.959280,4.959280,2.645552,[[ 4.29727096 1.75201314 1.84805634 0. ...
4,calc_LAMMPS_ELASTIC,2009--Purja-Pun-G-P--Ni-Al,C1--CaF2--fluorite,Al2Ni,-3.914698,5.638257,5.638257,5.638257,[[ 0.87728462 0.5238041 0.5238041 0. ...
5,calc_LAMMPS_ELASTIC,2012--Proville-L--Fe,A1--Cu--fcc,Fe,-4.000450,3.656868,3.656868,3.656868,[[ 1.02714725 0.80051106 0.80051106 0. ...
6,calc_refine_structure,2015--Wilson-S-R--Na,A3--Mg--hcp,Na,-8.352723,2.143546,3.712730,3.496358,[[ 26.76584295 24.04576559 23.74419709 0. ...
7,calc_refine_structure,2012--Park-H--Mo,A2--W--bcc,Mo,-0.403480,5.210851,5.210851,5.210851,[[ 0.20204241 0.23303331 0.23303331 0. ...
8,calc_refine_structure,2009--Purja-Pun-G-P--Ni-Al,Ah--alpha-Po--sc,Ni,-3.726400,2.396501,2.396501,2.396501,[[ 1.3938499 0.14888128 0.14888128 0. ...
9,calc_refine_structure,2009--Purja-Pun-G-P--Ni-Al,A15--beta-W,Al,-3.346935,5.146685,5.146685,5.146685,[[ 1.29050757 0.52315835 0.52315835 0. ...


## 3. HTML Tables

This section takes the processed data and generates per_potential html tables.

In [19]:
table_style_file = 'webtablestyle.html'

In [20]:
with open(table_style_file) as f:
    table_style = f.read() 

In [21]:
def gen_struct_table(df, potential, composition):
    
    headers = OrderedDict([ ('family', 'prototype'),
                            ('calculation', 'calculation'),
                            ('Ecoh', '<i>E</i><sub>coh</sub> (eV)'),
                            ('alat', '<i>a</i><sub>0</sub> (&Aring;)'),
                            ('blat', '<i>b</i><sub>0</sub> (&Aring;)'),
                            ('clat', '<i>c</i><sub>0</sub> (&Aring;)')])
    def float_fmt(value):
        return '%8.4f' % value
    
    table_df = df[(df.potential==potential) & (df.composition==composition)].sort_values('Ecoh')
    table_df = pd.DataFrame(table_df, columns=headers.keys())

    table_df.Ecoh = uc.get_in_units(table_df.Ecoh, 'eV')
    table_df.alat = uc.get_in_units(table_df.alat, 'Angstrom')
    table_df.blat = uc.get_in_units(table_df.blat, 'Angstrom')
    table_df.clat = uc.get_in_units(table_df.clat, 'Angstrom')
    
    table_df.rename(columns=headers, inplace=True)
    table_df.reset_index(drop=True, inplace=True)
    
    return table_df.to_html(index=False, float_format=float_fmt, escape=False, classes='datatable')
    
    
def gen_Cij_table(row):
    shift = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6}
    Cij = uc.get_in_units(row.C.Cij, 'GPa')
    C_df = pd.DataFrame(Cij)
    C_df = C_df.rename(index=shift, columns=shift)
    def float_fmt(value):
        return '%.2f' % value
    
    return C_df.to_html(float_format=float_fmt, escape=False, classes='datatable')

class ContentSelect(object):
    
    class Option(object):
        
        def __init__(self, value, data):
            self.__value = value
            self.__data = data
            
        @property
        def value(self):
            return self.__value
    
        @property
        def data(self):
            return self.__data
    
    def __init__(self, elementID, valuename='value', functionname='showSelect', selectID='pick'):
        self.__options = []
        self.__elementID = elementID
        self.__valuename = valuename
        self.__functionname = functionname
        self.__selectID = selectID
        
    def addoption(self, value, data):
        self.__options.append(self.Option(value, data))
        
    def genselect(self):
        html = '<select id="%s" onchange="%s()">\n' % (self.__selectID, self.__functionname)
        for i, option in enumerate(self.__options):
            if i > 0:
                html += '  <option>%s</option>\n' % (option.value)
            else:
                html += '  <option selected>%s</option>\n' % (option.value)       
        html += '</select>\n'
        
        return html

    def genswitch(self):
        html = '<script>\n'
        html += 'function %s() {\n' % self.__functionname
        html += '  var text;\n'
        html += '  var %s = document.getElementById("%s").value;\n' % (self.__valuename, self.__selectID)
        html += '  switch(%s) {\n' % (self.__valuename)
        for option in self.__options:
            html += '    case "%s":\n' % option.value
            html += "      text = '%s';\n" % option.data.replace('\n', '\\n')
            html += '      break;\n'
        html += '  }\n' 
        html += '  document.getElementById("%s").innerHTML = text;\n' % self.__elementID
        html += '}\n'
        html += '</script>\n'
        
        return html
    
    def gendiv(self):
        html = '<div id="%s">' % self.__elementID
        html += self.__options[0].data
        html += '</div>\n'
        
        return html

In [22]:
test = ContentSelect('test')

In [23]:
for i in xrange(5):
    test.addoption(df.iloc[i].family, '<i>C<sub>ij</sub></i> (GPa) = ' + gen_Cij_table(df.iloc[i]))

In [24]:
print(test.genselect()+test.gendiv()+test.genswitch())

<select id="pick" onchange="showSelect()">
  <option selected>B3--ZnS--cubic-zinc-blende</option>
  <option>A7--alpha-As</option>
  <option>A3'--alpha-La--double-hcp</option>
  <option>A5--beta-Sn</option>
  <option>C1--CaF2--fluorite</option>
</select>
<div id="test"><i>C<sub>ij</sub></i> (GPa) = <table border="1" class="dataframe datatable">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>1</th>
      <th>2</th>
      <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>24.36</td>
      <td>53.32</td>
      <td>53.32</td>
      <td>0.00</td>
      <td>0.00</td>
      <td>0.00</td>
    </tr>
    <tr>
      <th>2</th>
      <td>53.32</td>
      <td>24.36</td>
      <td>53.32</td>
      <td>0.00</td>
      <td>0.00</td>
      <td>0.00</td>
    </tr>
    <tr>
      <th>3</th>
      <td>53.32</td>
      <td>53.32</td>
      <td>24.36</td>
      <td>0.00</td>
      <td>0.00</td>
      <td>0.

In [25]:
display(HTML(test.genselect()+test.gendiv()+test.genswitch()))

,1,2,3,4,5,6
1,24.36,53.32,53.32,0.00,0.00,0.00
2,53.32,24.36,53.32,0.00,0.00,0.00
3,53.32,53.32,24.36,0.00,0.00,0.00
4,0.00,0.00,0.00,38.69,0.00,0.00
5,0.00,0.00,0.00,0.00,38.69,0.00
6,0.00,0.00,0.00,0.00,0.00,38.69


In [26]:
display(HTML(table_style+gen_Cij_table(df.iloc[0])))

,1,2,3,4,5,6
1,24.36,53.32,53.32,0.00,0.00,0.00
2,53.32,24.36,53.32,0.00,0.00,0.00
3,53.32,53.32,24.36,0.00,0.00,0.00
4,0.00,0.00,0.00,38.69,0.00,0.00
5,0.00,0.00,0.00,0.00,38.69,0.00
6,0.00,0.00,0.00,0.00,0.00,38.69


In [32]:
display(HTML(table_style+gen_struct_table(df, '2009--Purja-Pun-G-P--Ni-Al', 'AlNi3')))

prototype,calculation,Ecoh (eV),a0 (Å),b0 (Å),c0 (Å)
L1_2--AuCu3,calc_LAMMPS_ELASTIC,-4.6315,3.5332,3.5332,3.5332
L1_2--AuCu3,calc_refine_structure,-4.6315,3.5332,3.5332,3.5332
D0_3--BiF3,calc_refine_structure,-4.5988,5.5425,5.5425,5.5425
D0_3--BiF3,calc_LAMMPS_ELASTIC,-4.5988,5.5425,5.5425,5.5425
A15--Cr3Si,calc_refine_structure,-4.5532,4.4363,4.4363,4.4363
A15--Cr3Si,calc_LAMMPS_ELASTIC,-4.5532,4.4363,4.4363,4.4363


In [28]:
killit

NameError: name 'killit' is not defined

In [ ]:
t = test.iloc[0]

In [ ]:
2**0.5/2

In [ ]:
t.clat/t.alat

### 3.1 Parameters

In [ ]:
per_potential_directory = 'C:\\Users\\lmh1\\Documents\\website\\per_potential'

### 3.2 Data conversion parameters

__headers__ gives the list of data columns from data to include in and how they should be renamed in html_data.

In [ ]:
headers = OrderedDict([
        ('prototype',   'prototype'),
        ('Ecoh (eV)',   '<i>E</i><sub>coh</sub> (eV)'),
        ('a (A)',       '<i>a</i><sub>0</sub> (&Aring;)'),
        ('b (A)',       '<i>b</i><sub>0</sub> (&Aring;)'),
        ('c (A)',       '<i>c</i><sub>0</sub> (&Aring;)'),
        ('C11 (GPa)',   '<i>C</i><sub>11</sub> (GPa)'),
        ('C22 (GPa)',   '<i>C</i><sub>22</sub> (GPa)'),
        ('C33 (GPa)',   '<i>C</i><sub>33</sub> (GPa)'),
        ('C12 (GPa)',   '<i>C</i><sub>12</sub> (GPa)'),
        ('C13 (GPa)',   '<i>C</i><sub>13</sub> (GPa)'),
        ('C23 (GPa)',   '<i>C</i><sub>23</sub> (GPa)'),
        ('C44 (GPa)',   '<i>C</i><sub>44</sub> (GPa)'),
        ('C55 (GPa)',   '<i>C</i><sub>55</sub> (GPa)'),
        ('C66 (GPa)',   '<i>C</i><sub>66</sub> (GPa)') ])

__formating__ gives the c-style print format to use for the indivdual floating point terms

In [ ]:
l_const_format = '{:.4f}'
eng_coh_format = '{:.4f}'
e_const_format = '{:.2f}'

def formatter(style, value):
    if pd.notnull(value):
        return style.format(value)
    else:
        return ''

formatters = {'<i>E</i><sub>coh</sub> (eV)':    lambda x: formatter(eng_coh_format, x),
              '<i>a</i><sub>0</sub> (&Aring;)': lambda x: formatter(l_const_format, x),
              '<i>b</i><sub>0</sub> (&Aring;)': lambda x: formatter(l_const_format, x),
              '<i>c</i><sub>0</sub> (&Aring;)': lambda x: formatter(l_const_format, x),
              '<i>C</i><sub>11</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>22</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>33</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>12</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>13</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>23</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>44</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>55</sub> (GPa)':    lambda x: formatter(e_const_format, x),
              '<i>C</i><sub>66</sub> (GPa)':    lambda x: formatter(e_const_format, x)}

### 3.3 Other HTML content 

Here is where additional content of the resulting html file is collected.

In [ ]:
html_style = """
<style>
    .datatable {
        border: 1px solid black; 
        border-collapse: collapse; 
        padding: 5px; 
        text-align: right;

    } 
    .datatable td {
        border: 1px solid black; 
        border-collapse: collapse; 
        font: "Courier New", monospace; 
        font-size: 12px; 
        padding: 5px; 
        text-align: right;
        width: 45px;
    }
    .datatable td:nth-child(1) {
        width: 135px;
        text-align: left;
    }
    .datatable th {
        border: 1px solid black; 
        border-collapse: collapse; 
        font: "Courier New", monospace; 
        font-size: 12px; 
        padding: 5px; 
        text-align: left;
    }
</style>
"""

In [ ]:
html_info = """
<h2>Static Crystal Structure Predictions</h2>
<p>
    The properties listed here are obtained from static calculations for given 
    crystal structures. The values were obtained using an algorithm that takes 
    an initial estimate for the lattice constants and evaluates the cohesive 
    energy and virial pressures for the structure. Elastic constants are calculated 
    using the changes in the virial pressures due to the application of small strains 
    (1e-5). The pressure values and elastic compliances are used to obtain a new lattice 
    parameter guess by linearly extrapolating to zero pressure. This process is repeated 
    until the lattice constants from one iteration to the next are within a relative 
    tolerance of 1e-10. The elastic constants shown coincide with the final iteration.
</p><p>
    Initial estimates for the lattice constants correspond to all the energy minima 
    identified in the cohesive energy vs interatomic spacing plots. This means that 
    it is possible that some potentials have multiple refined results for the same 
    crystal structure. Having multiple energy minimums for a structure does not 
    necessarily make the potential 'bad' as unwanted configurations may be unstable or 
    correspond to conditions that may not be relevant to the problem of interest 
    (eg. very high strains).
</p><p>
    More information about the calculation used can be found on the 
    <a href="http://www.ctcms.nist.gov/potentials/tools.html">Tools</a> page.
</p><p>
    <a href="http://www.nist.gov/public_affairs/disclaimer.cfm">NIST disclaimer</a>
</p><p>
    <b>Disclaimer:</b> These values are meant to be guidelines for comparing 
    potentials, not the absolute values for any potential's properties. The 
    presence of any structures in this list does not guarantee that those 
    structures are stable as only the box dimensions are changed, not the 
    relative positions of the atoms in the cell. Also, the lowest energy 
    structure may not be included in this list. Variations in the values may 
    occur for fully relaxed configurations, different small strain values, 
    different simulation software and different implementations of the 
    interatomic potential. The algorithm used works best when the interatomic 
    potential's elastic constants vary smoothly with changes in volume.
</p><p>
    <b>Version Information:</b> As property calculation methods are developed and updated, there 
    may be changes in the calculated values. Updates to the calculation methods 
    that affect the values will be documented and archival versions of this page 
    will be made available as a record. 
    <ul><li>
        2016-09-28. Values for simple compounds added. All identified energy minima 
        for each structure are listed. The existing elemental data was regenerated. Most values are 
        consistent with before, but some differences have been noted. Specifically, variations are 
        seen with some values for potentials where the elastic constants don't vary smoothly near 
        the equilibrium state. Additionally, the inclusion of some high-energy structures has 
        changed based on new criteria for identifying when structures have relaxed to another structure.        
    </li><li>
        2016-04-07. Values for elemental crystal structures added. Only values for the 
        global energy minimum of each unique structure given.
    </li></ul>
</p>
<hr/>
"""

In [ ]:
html_note = '*<i>Multiple values for the same structure are due to multiple energy minima. More information in calculation description.</i>'

### 3.4 Code

In [ ]:
for potential in np.unique(data.potential):
    potential_data = data[data.potential==potential]
    
    html = html_style + html_info
    
    #Check that a directory exists for the potential
    if not os.path.isdir(os.path.join(per_potential_directory, potential)):
        os.makedirs(os.path.join(per_potential_directory, potential))
    
    for composition in np.unique(potential_data.composition):
        composition_data = potential_data[potential_data.composition == composition]
        html += '<h3>0K Crystal Structure Properties for ' + composition + '</h3>\n'
        
        html_data = pd.DataFrame(potential_data[potential_data.composition==composition], columns=headers.keys())
        html_data.rename(columns=headers, inplace=True)
        html_data.reset_index(drop=True, inplace=True)
        
        prototypes, pcounts = np.unique(html_data.prototype, return_counts=True)
        note = False
        for prototype, pcount in zip(prototypes, pcounts):
            if pcount > 1:
                print potential, composition, prototype
                html_data.prototype.loc[html_data.prototype==prototype] = prototype+'*'
                note = True
        
        html_data.sort_values('<i>E</i><sub>coh</sub> (eV)', inplace=True)
        
        html += html_data.to_html(index=False, escape=False, formatters=formatters, classes='datatable') +'\n'
        
        if note:
            html += html_note
        html += '<hr/>\n'
        
    with open(os.path.join(per_potential_directory, potential, 'struct.info'), 'w') as html_file:
        html_file.write(html)

## 4. Comparison Plots